<a href="https://colab.research.google.com/github/dmcguire81/metapy/blob/task%2Fgoogle_colab/tutorials/5-topic-modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
# NOTE: this assumes you've uploaded a Python 3.7 build from our fork to Drive
# TODO: replace this with a stock install when it's published somewhere
%pip install /content/drive/MyDrive/metapy-0.2.13-cp37-cp37m-manylinux_2_24_x86_64.whl

First, let's import the Python bindings, as usual.

In [3]:
import metapy

In [4]:
metapy.__version__ # you will want your version to be >= to this

'0.2.13'

If you would like to, you can inform MeTA to output log data to stderr like so:

In [5]:
# Uncomment for detailed (but installation-specific) progress logs
# metapy.log_to_stderr()

Now, let's download a list of stopwords and a sample dataset to begin exploring MeTA's topic models.

In [6]:
%%capture
!wget -N https://raw.githubusercontent.com/meta-toolkit/meta/master/data/lemur-stopwords.txt

In [7]:
%%capture
!wget -N https://meta-toolkit.org/data/2016-01-26/ceeaus.tar.gz
!tar xf ceeaus.tar.gz

We will need to index our data to proceed. We eventually want to be able to extract the bag-of-words representation for our individual documents, so we will want a `ForwardIndex` in this case.

In [8]:
%%capture
!wget -N https://raw.githubusercontent.com/dmcguire81/metapy/master/tutorials/ceeaus-config.toml

In [9]:
fidx = metapy.index.make_forward_index('ceeaus-config.toml')

Just like in classification, the feature set used for the topic modeling will be the feature set used at the time of indexing, so if you want to play with a different set of features (like bigram words), you will need to re-index your data.

For now, we've just stuck with the default filter chain for unigram words, so we're operating in the traditional bag-of-words space.

Let's load our documents into memory to run the topic model inference now.

In [10]:
dset = metapy.learn.Dataset(fidx)

Now, let's try to find some topics for this dataset. To do so, we're going to use a generative model called a topic model.

There are many different topic models in the literature, but the most commonly used topic model is Latent Dirichlet Allocation. Here, we propose that there are K topics (represented with a categorical distribution over words) $\phi_k$ from which all of our documents are genereated. These K topics are modeled as being sampled from a Dirichlet distribution with parameter $\vec{\alpha}$. Then, to generate a document $d$, we first sample a distribution over the K topics $\theta_d$ from another Dirichlet distribution with parameter $\vec{\beta}$. Then, for each word in this document, we first sample a topic identifier $z \sim \theta_d$ and then the word by drawing from the topic we selected ($w \sim \phi_z$). Refer to the [Wikipedia article on LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) for more information.

The goal of running inference for an LDA model is to infer the latent variables $\phi_k$ and $\theta_d$ for all of the $K$ topics and $D$ documents, respectively. MeTA provides a number of different inference algorithms for LDA, as each one entails a different set of trade-offs (inference in LDA is intractable, so all inference algorithms are approximations; different algorithms entail different approximation guarantees, running times, and required memroy consumption). For now, let's run a Variational Infernce algorithm called CVB0 to find two topics. (In practice you will likely be finding many more topics than just two, but this is a very small toy dataset.)

In [11]:
lda_inf = metapy.topics.LDACollapsedVB(dset, num_topics=2, alpha=1.0, beta=0.01)
lda_inf.run(num_iters=1000)

The above ran the CVB0 algorithm for 1000 iterations, or until an algorithm-specific convergence criterion was met. Now let's save the current estimate for our topics and topic proportions.

In [12]:
lda_inf.save('lda-cvb0')

We can interrogate the topic inference results by using the `TopicModel` query class. Let's load our inference results back in.

In [13]:
model = metapy.topics.TopicModel('lda-cvb0')

Now, let's have a look at our topics. A typical way of doing this is to print the top $k$ words in each topic, so let's do that.

In [14]:
model.top_k(tid=0)

[(3759, 0.06705632876205436),
 (1968, 0.05605926723261055),
 (2635, 0.052223032543735365),
 (3549, 0.04642935755269033),
 (665, 0.0348813869181328),
 (4157, 0.029067464231632915),
 (2322, 0.028850202852809672),
 (3729, 0.022331330087063384),
 (1790, 0.020755684552465195),
 (3554, 0.015483026545578923)]

The models operate on term ids instead of raw text strings, so let's convert this to a human readable format by using the vocabulary contained in our `ForwardIndex` to map the term ids to strings.

In [15]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0)]

[('time', 0.06705632876205436),
 ('job', 0.05605926723261055),
 ('part', 0.052223032543735365),
 ('student', 0.04642935755269033),
 ('colleg', 0.0348813869181328),
 ('work', 0.029067464231632915),
 ('money', 0.028850202852809672),
 ('think', 0.022331330087063384),
 ('import', 0.020755684552465195),
 ('studi', 0.015483026545578923)]

In [16]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1)]

[('smoke', 0.13110405053979196),
 ('restaur', 0.054349353543834446),
 ('peopl', 0.03678011107867773),
 ('smoker', 0.03349266037541876),
 ('ban', 0.022530687474535425),
 ('think', 0.015620499600262804),
 ('japan', 0.0127809265376863),
 ('complet', 0.012635077394220635),
 ('cigarett', 0.011987190618018648),
 ('non', 0.011317747304595558)]

We can pretty clearly see that this particular dataset was about two major issues: smoking in public and part time jobs for students. This dataset is actually a collection of essays written by students, and there just so happen to be two different topics they can choose from!

The topics are pretty clear in this case, but in some cases it is also useful to score the terms in a topic using some function of the probability of the word in the topic and the probability of the word in the other topics. Intuitively, we might want to select words from each topic that best reflect that topic's content by picking words that both have high probability in that topic **and** have low probability in the other topics. In other words, we want to balance between high probability terms and highly specific terms (this is kind of like a tf-idf weighting). One such scoring function is provided by the toolkit in `BLTermScorer`, which implements a scoring function proposed by Blei and Lafferty.

In [17]:
scorer = metapy.topics.BLTermScorer(model)
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=0, scorer=scorer)]

[('job', 0.34822034178481953),
 ('part', 0.313110533525486),
 ('student', 0.2832891581909065),
 ('colleg', 0.20808984925341356),
 ('time', 0.17796723381494317),
 ('money', 0.16234670384226768),
 ('work', 0.1558539350348198),
 ('studi', 0.08228285083986764),
 ('learn', 0.06491899371071641),
 ('experi', 0.054945232607250816)]

In [18]:
[(fidx.term_text(pr[0]), pr[1]) for pr in model.top_k(tid=1, scorer=scorer)]

[('smoke', 0.8741650110576629),
 ('restaur', 0.31746102931732345),
 ('smoker', 0.20060288327156517),
 ('ban', 0.12853045561809617),
 ('cigarett', 0.06557610093408339),
 ('non', 0.06128426198079873),
 ('complet', 0.06105378146115514),
 ('japan', 0.05846477556972651),
 ('health', 0.05054837802522116),
 ('seat', 0.045339924868740056)]

Here we can see that the uninformative word stem "think" was downweighted from the word list from each topic, since it had relatively high probability in either topic.

We can also see the inferred topic distribution for each document.

In [19]:
model.topic_distribution(0)

<metapy.stats.Multinomial {0: 0.978659, 1: 0.021341}>

It looks like our first document was written by a student who chose the part-time job essay topic...

In [20]:
model.topic_distribution(900)

<metapy.stats.Multinomial {0: 0.021203, 1: 0.978797}>

...whereas this document looks like it was written by a student who chose the public smoking essay topic.

We can also infer topics for a brand new document. First, let's create the document and use the forward index we loaded before to convert it to a feature vector:

In [21]:
doc = metapy.index.Document()
doc.content("I think smoking in public is bad for others' health.")
fvec = fidx.tokenize(doc)

Now, let's load a topic model inferencer that uses the same CVB inference method we used earlier:

In [22]:
inferencer = metapy.topics.CVBInferencer('lda-cvb0.phi.bin', alpha=1.0)

Now, let's infer the topic proportions for the new document:

In [23]:
proportions = inferencer.infer(fvec, max_iters=20, convergence=1e-4)
print(proportions)

<metapy.stats.Multinomial {0: 0.185608, 1: 0.814392}>
